In [1]:
import ibmJupyterNotebookStyles
ibmJupyterNotebookStyles.apply_ibm_styles()

## Get started with variational quantum linear solver

### Background

The variational quantum linear solver (VQLS), as originally proposed in Bravo-Prieto et al. 2020<sup>1</sup>, is a quantum algorithm that uses a variational scheme to solve systems of linear equations. 
Specifically, if we are given some matrix $\textbf{A}$, such that $\textbf{A} \textbf{x} \ = \ \textbf{b}$, where 
$\textbf{b}$ is some known vector, the VQLS algorithm attempts to find $\textbf{x}$ that is proportional to the 
actual solution. Such an algorithm can be run on near-term devices, and recent studies indicate that VQLS has better 
runtime scaling (empirically obtained) compared to the classical schemes.<sup>1</sup>

### Algorithm

Let’s first make some assumptions about the form of $\textbf{A}$ and $\textbf{b}$. The first assumption is that matrix $\textbf{A}$ is Hermitian and $A \in \mathbb{C}^{N \times N}$ is given to us as a linear combination of unitaries, that is, $A = \sum_{i=0}^{m-1} c_i A_i$, where $c_i$ are real numbers and each $A_i$ is a unitary. One such form of decomposition can be realized by expressing $A$ as a linear combination of Pauli strings (used later in the tutorial). Also, we assume that the vector $\textbf{b}=\{b_0, b_1, b_2, \cdots, b_{N-1}\}$, where $c_i$ are real numbers and each $A_i$ is a unitary. Furthermore, we assume that the vector $\textbf{b}=(b_0, b_1, b_2, \cdots, b_{N-1})$ can be loaded to a quantum state $\lvert b \rangle = \frac{1}{\sum_{j=0}^{N-1} \lvert b_i \rvert^2} \sum_{i=0}^{N-1} b_i \lvert i \rangle$ using a suitable unitary $U$. 
Hence:

$$\lvert b \rangle = U \lvert 0 \rangle$$

Now, the goal is to prepare a quantum state $\lvert x\rangle$ such that $A\lvert x\rangle = \lvert b \rangle$. As a step toward finding $\lvert x\rangle$, the algorithm prepares a trial state using a parameterized unitary $V(\boldsymbol{\theta}).$ Hence:

$$\lvert \psi(\boldsymbol{\theta}) \rangle = V(\boldsymbol{\theta}) \lvert 0 \rangle$$

The goal then is to find parameters $\boldsymbol{\theta}$ such that the distance between the state $A\lvert \psi(\boldsymbol{\theta}) \rangle$ and $\lvert b \rangle$ is minimized. One such measure of the distance between quantum states is the fidelity measure $F$, which for two pure states $\lvert a \rangle$ and $\lvert b \rangle$ is defined as $\lvert \langle a \lvert b \rangle \rvert^2$. Note that because $A$ is not a unitary, the norm of $A\lvert \psi(\boldsymbol{\theta}) \rangle$ is not expected to be a unitary. As a result, we would work with $\frac{1}{\lvert A\lvert \psi(\boldsymbol{\theta}) \rangle \rvert^2}A\lvert \psi(\boldsymbol{\theta}) \rangle$. Hence, we would like to maximize $F$, or minimize $1 - F$. The steps below show how $1-F$ can be rewritten so that the terms involved can be computed using a quantum circuit.
$$
\begin{align}
1 - F(\boldsymbol{\theta}) &=  1 - \frac{1}{\lvert A\lvert \psi(\boldsymbol{\theta}) \rangle \rvert^2}\lvert 
(A\lvert \psi(\boldsymbol{\theta}) \rangle)^{\dagger} \lvert b \rangle \rvert^2 
\\
&= 1 - \frac{1}{\lvert A\lvert \psi(\boldsymbol{\theta}) \rangle \rvert^2}
\lvert \langle  \psi(\boldsymbol{\theta}) \rvert A^{\dagger} \lvert b \rangle \rvert^2
\\
&= 1 - \frac{1}{\lvert A\lvert \psi(\boldsymbol{\theta}) \rangle \rvert^2}
\lvert \langle  \psi(\boldsymbol{\theta}) \rvert A^{\dagger} U \lvert 0 \rangle \rvert^2
\\
&= 1 - \frac{1}{\lvert A\lvert \psi(\boldsymbol{\theta}) \rangle \rvert^2}
\lvert (V(\boldsymbol{\theta}) \lvert 0 \rangle)^{\dagger} A^{\dagger} U \lvert 0 \rangle \rvert^2
\end{align}
$$

As we have $A = \sum_{i=0}^{m-1} c_i A_i$, we can further rewrite the above expression as:
$$
\begin{align}
&= 1 - \frac{1}{\lvert A\lvert \psi(\boldsymbol{\theta}) \rangle \rvert^2}
\lvert \sum_{i=0}^{M-1} c_i \langle 0 \rvert V(\boldsymbol{\theta})^{\dagger} A_i^{\dagger} U \lvert 0 \rangle \rvert^2 
\\
&= 1 - \frac{1}{(A\lvert \psi(\boldsymbol{\theta}) \rangle)^{\dagger} A\lvert \psi(\boldsymbol{\theta}) \rangle}
\big( \sum_{i=0}^{M-1} c_i \langle 0 \rvert V(\boldsymbol{\theta})^{\dagger} A_i^{\dagger} U \lvert 0 \rangle \big)
\big(\sum_{j=0}^{N-1} c_j \langle 0 \rvert V(\boldsymbol{\theta})^{\dagger} A_j^{\dagger} U \lvert 0 \rangle\big)^{\dagger} 
\\
&= 1 - \frac{1}{\langle \psi(\boldsymbol{\theta})\rvert A^{\dagger} A\lvert \psi(\boldsymbol{\theta}) \rangle}
\sum_{i=0}^{M-1} \sum_{j=0}^{N-1} c_i c_j^{*} \langle 0 \rvert V(\boldsymbol{\theta})^{\dagger} A_i^{\dagger} U 
\lvert 0 \rangle \langle 0 \lvert U^{\dagger} A_j V(\boldsymbol{\theta}) \lvert 0 \rangle  
\end{align}
$$

#### Computation of terms

Now, looking at the above expression, let’s focus on the term in the numerator: $L(\boldsymbol{\theta}) 
= \langle 0 \rvert V(\boldsymbol{\theta})^{\dagger} A_i^{\dagger} 
U \lvert 0 \rangle \langle 0 \lvert U^{\dagger} A_j V(\boldsymbol{\theta}) \lvert 0 \rangle$. 

In the original paper,<sup>1</sup> the projection operator $\lvert 0 \rangle \langle 0 \lvert$ is substituted for another “local” operator:

$$
\frac{1}{2} + \frac{1}{2n}{\sum_{i=0}^{n-1} Z_i}
$$

This local version of the cost function leads to further simplification of the circuit (see below) and is expected to present a more amenable minimization landscape.<sup>1</sup> It leads to the fidelity expression:

$$
\begin{split}
1 - F_l(\boldsymbol{\theta}) &= 1 - \frac{1}{\langle \psi(\boldsymbol{\theta})\rvert A^{\dagger} A\lvert \psi(\boldsymbol{\theta}) 
                                             \rangle}
\sum_{i=0}^{M-1} \sum_{j=0}^{N-1} c_i c_j^{*} \langle 0 \rvert V(\boldsymbol{\theta})^{\dagger} A_i^{\dagger} U 
\big(\frac{1}{2} + \frac{1}{2n}{\sum_{k=0}^{n-1} Z_k}\big) U^{\dagger} A_j V(\boldsymbol{\theta}) \lvert 0 \rangle 
\\
&= 1 - \frac{1}{\langle \psi(\boldsymbol{\theta})\rvert A^{\dagger} A\lvert \psi(\boldsymbol{\theta}) \rangle}
\big(\frac{1}{2}\sum_{i=0}^{M-1} \sum_{j=0}^{N-1} c_i c_j^{*} \langle 0 \rvert 
V(\boldsymbol{\theta})^{\dagger} 
A_i^{\dagger} U U^{\dagger} A_j V(\boldsymbol{\theta}) \lvert 0 \rangle + \frac{1}{2n}\sum_{i=0}^{M-1} \sum_{j=0}^{N-1} 
     \sum_{k=0}^{n-1} c_i c_j^{*} \langle 0 \rvert 
V(\boldsymbol{\theta})^{\dagger} 
A_i^{\dagger} U Z_k U^{\dagger} A_j V(\boldsymbol{\theta}) \lvert 0 \rangle \big) \\
&= 1 - \frac{1}{\langle \psi(\boldsymbol{\theta})\rvert A^{\dagger} A\lvert \psi(\boldsymbol{\theta}) \rangle}
\big(\frac{1}{2} \langle \psi(\boldsymbol{\theta})\rvert A^{\dagger} A\lvert \psi(\boldsymbol{\theta}) \rangle 
     + \frac{1}{2n}\sum_{i=0}^{M-1} \sum_{j=0}^{N-1} \sum_{k=0}^{n-1} c_i c_j^{*} \langle 0 \rvert 
V(\boldsymbol{\theta})^{\dagger} 
A_i^{\dagger} U Z_k U^{\dagger} A_j V(\boldsymbol{\theta}) \lvert 0 \rangle \big) 
\\
&= \frac{1}{2} - 
\frac{1}{2n}\frac{\sum_{i=0}^{M-1} \sum_{j=0}^{N-1} \sum_{k=0}^{n-1} c_i c_j^{*} \langle 0 \rvert 
V(\boldsymbol{\theta})^{\dagger} 
A_i^{\dagger} U Z_k U^{\dagger} A_j V(\boldsymbol{\theta}) 
\lvert 0 \rangle}{\sum_{i=0}^{M-1} \sum_{j=0}^{N-1} c_i c_j^{*} \langle 0 \rvert 
V(\boldsymbol{\theta})^{\dagger} 
A_i^{\dagger} A_j V(\boldsymbol{\theta}) 
\lvert 0 \rangle} 
\\
\end{split} 
$$

Hence: 
$$
\boxed{
1 - F_l(\boldsymbol{\theta}) = \frac{1}{2} - 
\frac{1}{2n}\frac{\sum_{i=0}^{M-1} \sum_{j=0}^{N-1} \sum_{k=0}^{n-1} c_i c_j^{*} \langle 0 \rvert 
V(\boldsymbol{\theta})^{\dagger} 
A_i^{\dagger} U Z_k U^{\dagger} A_j V(\boldsymbol{\theta}) 
\lvert 0 \rangle}{\sum_{i=0}^{M-1} \sum_{j=0}^{N-1} c_i c_j^{*} \langle 0 \rvert 
V(\boldsymbol{\theta})^{\dagger} 
A_i^{\dagger} A_j V(\boldsymbol{\theta}) 
\lvert 0 \rangle}}
$$
      
The quantity in the numerator and the denominator in the expression above can also be seen as expectation values of the form $\langle \psi(\boldsymbol{\theta}) \rvert Q \lvert \psi(\boldsymbol{\theta}) \rangle$. Such expectation values can be computed using a Hadamard test, as described below and in Bravo-Prieto 2020.<sup>1</sup>

#### Hadamard test

The Hadamard test is a well-known method used to compute expectation values of the form $\langle \psi | U | \psi \rangle$, where $U$ and $|\psi\rangle$ denote some arbitrary unitary and quantum state, respectively. The circuit below is used to compute $Re(\langle \psi | U | \psi \rangle)$. A variant of this circuit can also be used to compute the $Img(\langle \psi | U | \psi \rangle)$.

<br><br>

<img src="./figs/hadamard_test.png" alt="alt text" width="800"/>

<br><br>

The probability of measuring the first qubit to be $0$ is equal to $\frac{1}{2} (1 \ + \ \text{Re}\langle U \rangle)$, and the probability of measuring $1$ is $\frac{1}{2} (1 \ - \ \text{Re}\langle U \rangle)$; as such, subtracting the two probabilities gives us $\text{Re} \langle U \rangle$. The matrices with which we will be dealing when we test this algorithm are completely real, so for this specific implementation $\text{Re} \langle U \rangle \ = \ \langle U \rangle$. 

Here is how the Hadamard test works. By the circuit diagram, we have as our general state vector:

<br>



$$\frac{|0\rangle \ + \ |1\rangle}{\sqrt{2}} \ \otimes \ |\psi\rangle \ = \ \frac{|0\rangle \ \otimes \ |\psi\rangle \ + \ |1\rangle \ \otimes \ |\psi\rangle}{\sqrt{2}}$$



<br>

Applying our controlled unitary, we get:

<br>



$$\frac{|0\rangle \ \otimes \ |\psi\rangle \ + \ |1\rangle \ \otimes \ |\psi\rangle}{\sqrt{2}} \ \rightarrow \ \frac{|0\rangle \ \otimes \ |\psi\rangle \ + \ |1\rangle \ \otimes \ U|\psi\rangle}{\sqrt{2}}$$



<br>

Then applying the Hadamard gate to the first qubit, we get:

<br>



$$\frac{|0\rangle \ \otimes \ |\psi\rangle \ + \ |1\rangle \ \otimes \ U|\psi\rangle}{\sqrt{2}} \ \rightarrow \ \frac{1}{2} \ \big[ |0\rangle \ \otimes \ |\psi\rangle \ + \ |1\rangle \ \otimes \ |\psi\rangle \ + \ |0\rangle \ \otimes \ U|\psi\rangle \ - \ |1\rangle \ \otimes \ U|\psi\rangle \big]$$



<br>



$$\Rightarrow \ |0\rangle \ \otimes \ (I \ + \ U)|\psi\rangle \ + \ |1\rangle \ \otimes \ (I \ - \ U)|\psi\rangle$$



<br>

When we take a measurement of the first qubit, remember that to find the probability of measuring $0$, we must take the inner product of the state vector with $|0\rangle$, then multiply it by its complex conjugate. The same follows for the probability of measuring $1$. Thus, we have:

<br>



$$P(0) \ = \ \frac{1}{4} \ \langle \psi | (I \ + \ U) (I \ + \ U^{\dagger}) |\psi\rangle \ = \ \frac{1}{4} \ \langle \psi | (I^2 \ + U \ + \ U^{\dagger} \ + \ U^{\dagger} U) |\psi\rangle \ = \ \frac{1}{4} \ \langle \psi | (2I \ + U \ + \ U^{\dagger}) |\psi\rangle$$



<br>



$$\Rightarrow \ \frac{1}{4} \Big[ 2 \ + \ \langle \psi | U^{\dagger} | \psi \rangle \ + \ \langle \psi | U | \psi \rangle \Big] \ = \ \frac{1}{4} \Big[ 2 \ + \ (\langle \psi | U | \psi \rangle)^{*} \ + \ \langle \psi | U | \psi \rangle \Big] \ = \ \frac{1}{2} (1 \ + \ \text{Re} \ \langle \psi | U | \psi \rangle)$$



<br>

By a similar procedure, we get:

<br>



$$P(1) \ = \ \frac{1}{2} \ (1 \ - \ \text{Re} \ \langle \psi | U | \psi \rangle)$$



<br>

And so, by taking the difference, we get:

<br>



$$P(0) \ - \ P(1) \ = \ \text{Re} \ \langle \psi | U | \psi \rangle$$

<br>

#### Circuit implementations

Now let’s discuss the actual implementation of the Hadamard test in the current settings. As discussed before, we want to compute the numerator and the denominator in the expression of the fidelity derived above. 

$$
\boxed{
1 - F_l(\boldsymbol{\theta}) = \frac{1}{2} - 
\frac{1}{2n}\frac{\sum_{i=0}^{M-1} \sum_{j=0}^{N-1} \sum_{k=0}^{n-1} c_i c_j^{*} \langle 0 \rvert 
V(\boldsymbol{\theta})^{\dagger} 
A_i^{\dagger} U Z_k U^{\dagger} A_j V(\boldsymbol{\theta}) 
\lvert 0 \rangle}{\sum_{i=0}^{M-1} \sum_{j=0}^{N-1} c_i c_j^{*} \langle 0 \rvert 
V(\boldsymbol{\theta})^{\dagger} 
A_i^{\dagger} A_j V(\boldsymbol{\theta}) 
\lvert 0 \rangle}}
$$

#### Computation of the numerator
Each term in the numerator can be obtained by a Hadamard test circuit. Here we show the circuit implementation that would allow for the computation of the real part of the term. Another circuit would be needed to compute the imaginary part.

<img src="./figs/vqls.png" alt="alt text" width="1000"/>

This circuit can be further simplified by $\text{c-}U^{\dagger}\text{c-}Z\text{c-}U = U^{\dagger} (\text{c-}Z) U$. This simplification is beneficial because it removes two control gates that are costly to implement in hardware. This trick of dropping the controls on $U$ and $U^{\dagger}$ is allowed because $U U^{\dagger} = I$ in the simplified circuit, which is equivalent to the control qubit $q_0$ acting in the zero state $\vert 0 \rangle$ in the prior circuit. Hence, the circuit above can be further simplified as:

<img src="./figs/vqls-simple.png" alt="alt text" width="1000"/>

#### Computation of the denominator

Each term in the denominator can be computed by a circuit of the form shown below. Again, we show only the circuit for obtaining the real part of each term.

<img src="./figs/denominator.png" alt="alt text" width="700"/>

### Toy example: Coding an implementation in Qiskit

Let’s define a toy problem we would implement in Qiskit. Here we consider solving $A\lvert x \rangle = \lvert b \rangle$, where $A = c_0 A_0 + c_1 A_1$, $A_0 = I \otimes I \otimes I$, and 
$A_1 = Z \otimes X \otimes I.$ We consider:


$$ \lvert b \rangle = H^{\otimes 3} \lvert 000 \rangle$$

The construct discussed above can be used to compute fidelities using Hadamard tests. But for matrix $A$, which can be expressed as $M$ unitaries, ~$M^2 (n + 1)$ circuits are needed. To reduce the number of circuits from what is required in the original VQLS scheme, we consider an alternative scheme to implement $A$.

Instead of implementing separate circuits for each of the unitaries in the decomposition of $A$, as was done in Bravo-Prieto 2020<sup>1</sup>, we implement a single circuit here. This can be accomplished, as described below, by using an ancilla qubit that allows rotations to encode the coefficients $c_i$ in the linear combination of $A = \sum_i c_i A_i$.

We work it out below for our toy model problem, where $A = c_0 A_0 + c_1 A_1$. However, such a construct can easily be generalized to a sum involving more than two terms.<sup>2</sup>

Let $A = c_0 A_0 + c_1 A_1$, where $A_0$ and $A_1$ are unitaries. We introduce $\zeta$ so that $\cos(\zeta) = \sqrt{\frac{c_0}{|c_0| + |c_1|}}$ and $\sin(\zeta) = \sqrt{\frac{c_1}{|c_0| + |c_1|}}$. The denominator is the L1 norm $|c_0| + |c_1|$.

 <img src="./figs/lcu.png" alt="alt text" width="1200"/>

The set of transformations (shown above) we plan to do are:

$$
\begin{aligned}
& |0\rangle V(\theta)|0\rangle \mapsto R_Y(\zeta) |0\rangle V(\theta) |0\rangle \\
& \mapsto \bigg(\cos(\zeta)|0\rangle + \sin(\zeta) |1\rangle\bigg) V(\theta)|0\rangle \\
& \mapsto  \cos(\zeta) |0\rangle A_{0} V(\theta)|0\rangle 
+ \sin(\zeta) |1\rangle A_{1} V(\theta)|0\rangle  \\
& \mapsto |0 \rangle \frac{c_0}{|c_0| + |c_1|} \big( c_0 A_{0} + c_1 A_{1} \big) V(\theta)|0\rangle 
- |1\rangle \frac{\sqrt{c_0 c_1}} {|c_0| + |c_1|} \big(A_{0} - A_{1} \big) V(\theta) |0\rangle 
\end{aligned}
$$

where $R_Y(\zeta) = e^{-i \zeta Y}$.

It is clear that $A \lvert \psi(\theta) \rangle$ is implemented on the second register up to a multiplicative factor. 

In the next step, we would hit the bottom register with $U^{\dagger}$. Recall that $U \lvert 0 \rangle = \lvert b \rangle.$ Hence, $U^{\dagger} \lvert b \rangle = \lvert 0 \rangle$. The resulting state now is:

$$
\mapsto |0\rangle \frac{c_0}{|c_0| + |c_1|} U^{\dagger} \big(c_0 A_{0} + c_1 A_{1} \big) V(\theta)|0\rangle 
- |1\rangle \frac{\sqrt{c_0 c_1}} {|c_0| + |c_1|} U^{\dagger} \big(A_{0} - A_{1} \big) V(\theta) |0\rangle
$$

The state left after measuring zero on the ancilla qubit is:

$$
  |\Psi(\theta)\rangle = U^{\dagger} \frac{c_0}{|c_0| + |c_1|} \big(c_0 A_{0} + c_1 A_{1} \big) V(\theta)|0\rangle 
$$


The probability of observing all zero states in another set of measurements on the second register is: 

$$
\begin{align}
P(0|\text{ancilla in zero state}) &=  \frac{c_0^2}{(|c_0| + |c_1|)^2} \langle 0 | V^{\dagger}(\theta)\bigg(c_0 A_{0}^{\dagger} 
+ c_1 A_{1}^{\dagger} \bigg) U | 0 \rangle \langle 0 | U^{\dagger}\bigg(c_0 A_{0} 
+ c_1 A_{1} \bigg) V(\theta)|0\rangle \\
&= \frac{c_0^2}{(|c_0| + |c_1|)^2} \langle \psi(\theta) | A^{\dagger} |b \rangle \langle b| A | \psi(\theta) \rangle \\ 
\end{align} 
$$

Hence, we can interpret the probability as the fidelity between states $A |\psi(\theta)\rangle$ and $|b\rangle$. Note that for this to succeed, we ideally need to create enough copies of the final state so that we measure a zero state on the ancilla qubit with some non-zero probability. 

The probability of failure—that is, the probability of observing “one” state on the ancilla qubit—is:

$$
\begin{split}
P(1) &= \frac{c_0 c_1}{(|c_0| + |c_1|)^2} \bigg \lVert \big(A_{0} - A_{1} \big) V(\theta) |0\rangle \bigg \rVert^2 \\
&\le \frac{c_0 c_1}{(|c_0| + |c_1|)^2} \bigg \lVert A_{0} - A_{1} \bigg \rVert^2 
\bigg \lVert V(\theta) |0\rangle \bigg \rVert^2 \\
&= \frac{c_0 c_1}{(|c_0| + |c_1|)^2} \bigg \lVert A_{0} - A_{1}\bigg \rVert^2 \\
&\le \frac{4 c_0 c_1}{(|c_0| + |c_1|)^2}
\end{split}
$$
Hence, one needs to run this scheme (create copies of the quantum state) more than $ \frac{(|c_0| + |c_1|)^2}{4 c_0 c_1}$ times to ensure that a zero state is observed on the ancilla qubit at least once. A more general treatment when $A = \sum_i c_i A_i$ can be found in Childs and Wiebe 2012.<sup>2</sup>

#### Qiskit code for the toy problem

In [2]:
from qiskit_aer import Aer
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.quantum_info.states import partial_trace
from qiskit.circuit import ParameterVector

import math as math
import numpy as np

In [3]:
# Define global parameters

# Number of system qubits
n_qubits = 3  

# Number of parameterized unitary circuits
num_alpha = 3 

# Addition of an ancillary qubit
tot_qubits = n_qubits+1  

# Seed for random number generator for the ansatz
rng_seed = 0  

# A matrix coefficient
init_cm = np.array([1, 0.2])

# Conversion of the coefficient
norm = np.linalg.norm(init_cm, 1)

c_norm = init_cm / norm
n00 = np.sqrt(init_cm[0]/norm) 
theta = np.arccos(n00)
n01 = np.sin(theta)
cnorm_list = [n00, n01]
sim_f = 'qasm_simulator'

#nshots, set 1000 more than minimum needed (see discussion above)
Nshots = 1000 + int((cnorm_list[0] ** 2 + cnorm_list[1] ** 2) ** 2 / 4 * cnorm_list[0] * cnorm_list[1])

#### Define the functions

In [4]:
def A_weight_circuits(total_qubits, theta, amp_swap):
    
    """
    Y-rotations to implement coefficients of A_0 and A_1
    
    total_qubits: int
            circuit width
    theta: float
        angle of rotation based on coefficients of A
    amp_swap: bool
        The amp_swap argument determines with the forward Ry gate 
            (amp_swap=0) or inverse Ry gate (amp_swap=1) is used.
    """
    
    ancilla_idx =  total_qubits -1
    qc_input = QuantumCircuit(total_qubits)

    if amp_swap == 0:
        qc_input.ry(-2 * theta, ancilla_idx)
    elif amp_swap == 1:
        qc_input.ry(2 * theta, ancilla_idx)
    
    return qc_input


def Cont_A(total_qubits, idx):
    
    """
    Implements A's components A_0 and A_1
    """
    
    ancilla_idx =  total_qubits - 1
    qc = QuantumCircuit(total_qubits)
    
    if idx == 0:        
        pass
    elif idx == 1:
        qc.cz(ancilla_idx, 0)
        qc.cx(ancilla_idx, 1)
    
    return qc


def fixed_ansatz(n_qubits, alpha):
    
    """
    Prepares an estimate of the solution state |x>
    """
    
    circ = QuantumCircuit(n_qubits)

    for idx, element in enumerate(alpha):
        circ.h(idx)
        circ.ry(element, idx)
        
    return circ


def vqls_block(n_qubits, theta, cnorm_list, sim_f, alpha):
    
    """
    Main VQLS quantum circuit block
    """
    
    total_qubits = n_qubits + 1
    
    qreg = QuantumRegister(total_qubits)
    creg = ClassicalRegister(total_qubits)
    
    qc = QuantumCircuit(qreg,creg)
    
   # ancilla circuilt for the coefficient
    qc_init = A_weight_circuits(tot_qubits,theta,0)
    qc.compose(qc_init, inplace = True)
    
    qc.barrier()

    # First circuit (1) : to generate /X> from ansatz
    qc_ansatz = fixed_ansatz(n_qubits, alpha)
    qc.compose(qc_ansatz, inplace = True)
    
    qc.barrier()
    
    # Second circuit  (2) 
    # Controlled application of the unitary component A_m of the problem matrix A.
    for m in range(0, len(cnorm_list)):
        qc_A = Cont_A(total_qubits, m)
        qc.compose(qc_A, inplace = True)
        qc.barrier()

    # Third circuit (3)
    qc_Ub = Ub(n_qubits)
    qc.compose(qc_Ub.inverse(), inplace = True)
    
    qc.barrier()
    
    # Inverse of weight cirucit
    qc_init_inv = A_weight_circuits(tot_qubits,theta,1)
    qc.compose(qc_init_inv, inplace = True)
    
    qc.barrier()
    
    if sim_f=="qasm":
        qc.measure([0,1,2,3],[0,1,2,3])
    
    return qc


def Ub(n_qubits):
    
    """
    Prepares state |b>
    """
    
    qc = QuantumCircuit(n_qubits)
    for idx in range(n_qubits):
        qc.h(idx)
    return qc


def measure(qc, weights, sim_f):
    
    """
    Gathers measurement counts
    """
    
    bind_dict = {}
    for key, idx in zip(qc.parameters, range(0, len(qc.parameters))):
        bind_dict[key] = weights[idx]
        
    final_cir = qc.assign_parameters(bind_dict)
    
    sim_f = sim_f
    
    
    if sim_f=="state":
    
        simulator = Aer.get_backend('statevector_simulator')
        result = simulator.run(final_cir, run_options={'seed_transpiler': 10}).result()

        state_vector = result.get_statevector(final_cir).data
        
        expected_value = state_vector[:1] * np.conj(state_vector[:1])
        expected_value = expected_value[0].real

        # Partial trace
        
        H_out = np.outer(state_vector, np.conj(state_vector))
        rho_sub = partial_trace(H_out, [0,1,2]).data
        ancilla_density_mat_diag = np.diag(rho_sub)

        rho_sub = partial_trace(H_out, [3]).data
        work_density_mat_diag = np.diag(rho_sub)

        ancilla_zeros = ancilla_density_mat_diag[0].real
        
        if ancilla_zeros < 10 ** (-5):
            
            ancilla_zeros += 10 ** (-5)
        
        
        work_zeros = work_density_mat_diag[0].real
        
        all_zeros= (state_vector[0] * np.conj(state_vector[0])).real
        
        state_info=state_vector
        
    else:
        simulator = Aer.get_backend('qasm_simulator')
        result = simulator.run(final_cir, run_options={'shots': n_shots}).result()
            

        # Initialize dictionary
        counts_statevector =result.get_counts(final_cir)
        
        state_info =counts_statevector 
        
        # Finding available states
        ancilla_zeros = sum([state_info[x] for x in state_info.keys() if x[0] =='0'])        

        try:
            ones = counts_statevector["1111"]
        except:
            ones = 0

        try:
            all_zeros = counts_statevector["0000"]
        except:
            all_zeros =0

        total = n_shots

    expected_value = all_zeros/ancilla_zeros
   
    return expected_value, state_info


def run_circuit(weights, theta, cnorm_list, sim_f):
    
    """
    Fetches fidelity
    """
    
    alpha = ParameterVector('alpha', n_qubits)
    circuit = vqls_block(n_qubits, theta,  cnorm_list =cnorm_list, sim_f=sim_f, alpha=alpha)
    expected_value, counts_statevector = measure(circuit, weights, sim_f)
    
    
    return expected_value, counts_statevector


def obj(weights, cnorm_list=None, sim_f='qasm_simulator'):
    """
    returns objective
    """
    sim_f="state"
    
    mu_real, counts_vector_r = run_circuit(weights, theta=theta, cnorm_list=cnorm_list, sim_f=sim_f)
    
    print("cost:", 1 - mu_real)
    
    return 1 - mu_real

#### Run the circuit

In [5]:
def answer_X_block(n_qubits, weights):
    
    qc = QuantumCircuit(n_qubits)
    qc_ansatz= fixed_ansatz(n_qubits, weights)
    qc.compose(qc_ansatz, inplace = True)  
    
    return qc


np.random.seed(rng_seed)
w = np.random.randn(n_qubits)

from scipy.optimize import minimize
options={'maxiter': 1000}
opt = minimize(obj, args=(cnorm_list, sim_f), x0=w, tol=1e-6, method="COBYLA")
weights = opt.x

# Prepare solution
circuit = answer_X_block(n_qubits, weights)
print(circuit)
simulator = Aer.get_backend('statevector_simulator')
result = simulator.run(circuit).result()
answer_X = result.get_statevector(circuit)

cost: 0.7446411231168404
cost: 0.9999359714044088
cost: 0.8916076827946844
cost: 0.9011259473832209
cost: 0.1865777334605837
cost: 0.06269790851298673
cost: 0.16653014079361994
cost: 0.18655429617186214
cost: 0.2898017388942634
cost: 0.03792552025055018
cost: 0.07134799690304994
cost: 0.021325721230608163
cost: 0.01662921955273844
cost: 0.00507537810874692
cost: 0.002685840049026611
cost: 0.006013207734307691
cost: 0.007016884236906162
cost: 0.0030287822311539925
cost: 0.006858592569436439
cost: 0.002089332583240955
cost: 0.0014026537894330504
cost: 0.0014658684292871982
cost: 0.0020576089179047408
cost: 0.0006154716637487567
cost: 0.00017795201320824905
cost: 0.00030000918727479675
cost: 0.0004123330964456162
cost: 7.25389940537946e-05
cost: 9.117338583863877e-05
cost: 1.9450014991928555e-05
cost: 5.225022402788326e-05
cost: 4.1717977610211676e-05
cost: 6.505245712995489e-06
cost: 6.222958366941533e-06
cost: 6.995418987854052e-06
cost: 3.0134959491112667e-05
cost: 6.950904400615521e-0

#### Check the answer

In [6]:
Id = np.identity(2)
Z = np.array([[1, 0], [0, -1]])
X = np.array([[0, 1], [1, 0]])

A_0 = np.kron(np.kron(Id, Id), Id)

A_1 = np.kron(Id,np.kron(X, Z))

A_num = c_norm[0] * A_0 + c_norm[1] * A_1

b = np.ones(8) / np.sqrt(8)

A_inv = np.linalg.inv(A_num)
x = np.dot(A_inv, b)

#### Compute the state overlap between the quantum solution and the actual solution

In [7]:
np.dot(x/np.linalg.norm(x), answer_X/np.linalg.norm(answer_X))

(0.9999999999999173+0j)

### References

1. <a id='Bravo-Prieto2020'></a>Carlos Bravo-Prieto et al. “Variational Quantum Linear Solver.” arXiv:1909.05820, June 2, 2020. https://arxiv.org/pdf/1909.05820.pdf.

2. <a id='Childs2012'></a>Andrew M. Childs and Nathan Wiebe. “Hamiltonian Simulation Using Linear Combinations of Unitary Operations.” arXiv:1202.5822, February 27, 2012. https://arxiv.org/pdf/1202.5822.pdf.

<span style="font-size:10pt; font-weight:bold;" class = "copyright-content"> &copy; Copyright IBM Corp. 2023 <br/>This content is categorized as an "existing" IBM asset as part of the IBM Quantum Accelerator contract. </span>